In [1]:
# A policy iteration approach for dynamic programming for clearing a path for an emergency vehicle on a discretization road segment

In [31]:
import numpy as np

In [40]:
# The number of cells (length) for the road segment
L = 10
# Number of non-EV in front of the EV:
N = 3

In [41]:
left_vehicle_coordinates = np.random.choice(L, N, replace=False)

In [42]:
left_vehicle_coordinates = np.sort(left_vehicle_coordinates)

In [43]:
right_vehicle_coordinates = np.sort(np.random.choice(L, N, replace=False))
right_vehicle_coordinates

array([1, 6, 8])

In [52]:
# The right vehicle coordinates also serve as the environment information, indicating that vehicle on the left lane
# could not pull over into the right lane if there is a vehicle at the same coordinates:
obstacle_coordinates = right_vehicle_coordinates
obstacle_coordinates

array([1, 6, 8])

In [45]:
# Compile the starting state matrix:
flagships = np.zeros(shape = (N, 1))

In [51]:
state_matrix = np.concatenate((left_vehicle_coordinates.reshape(N,1), flagships), axis = 1)
state_matrix

array([[4., 0.],
       [6., 0.],
       [7., 0.]])

In [49]:
# Get the coordinate of the EV at current timestamp
def get_EV_coordinate(state_matrix):
    result = -1
    for row in state_matrix:
        if row[1] == 0:
            result = row[0]
    return result

In [50]:
def get_reward(state_matrix):
    if get_EV_coordinate(state_matrix) < 0:
        return 0
    else:
        return -1

7.0

In [ ]:
# Set up the policy matrix:


In [53]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)

In [ ]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI environment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """

    def one_step_lookahead(state, V):
        """
        Helper function to calculate the value for all action in a given state.
        
        Args:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.
        """
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        # Evaluate the current policy
        V = policy_eval_fn(policy, env, discount_factor)
        
        # Will be set to false if we make any changes to the policy
        policy_stable = True
        
        # For each state...
        for s in range(env.nS):
            # The best action we would take under the current policy
            chosen_a = np.argmax(policy[s])
            
            # Find the best action by one-step lookahead
            # Ties are resolved arbitarily
            action_values = one_step_lookahead(s, V)
            best_a = np.argmax(action_values)
            
            # Greedily update the policy
            if chosen_a != best_a:
                policy_stable = False
            policy[s] = np.eye(env.nA)[best_a]
        
        # If the policy is stable we've found an optimal policy. Return it
        if policy_stable:
            return policy, V